# Deep Learning
## Assignment 4
Previously in 2_fullyconnected.ipynb and 3_regularization.ipynb, we trained fully connected networks to classify notMNIST characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:

* convolutions need the image data formatted as a cube (width by height by #channels)
* labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [5]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Random Seed  
  tf.set_random_seed(42)  
    
  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [6]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.878395
Minibatch accuracy: 12.5%
Validation accuracy: 10.5%
Minibatch loss at step 50: 1.646402
Minibatch accuracy: 31.2%
Validation accuracy: 56.7%
Minibatch loss at step 100: 0.629248
Minibatch accuracy: 81.2%
Validation accuracy: 72.3%
Minibatch loss at step 150: 1.544412
Minibatch accuracy: 56.2%
Validation accuracy: 69.0%
Minibatch loss at step 200: 1.669333
Minibatch accuracy: 43.8%
Validation accuracy: 72.9%
Minibatch loss at step 250: 0.961547
Minibatch accuracy: 68.8%
Validation accuracy: 78.6%
Minibatch loss at step 300: 1.263107
Minibatch accuracy: 62.5%
Validation accuracy: 79.8%
Minibatch loss at step 350: 0.368872
Minibatch accuracy: 93.8%
Validation accuracy: 80.1%
Minibatch loss at step 400: 0.528980
Minibatch accuracy: 81.2%
Validation accuracy: 80.3%
Minibatch loss at step 450: 0.344425
Minibatch accuracy: 81.2%
Validation accuracy: 80.7%
Minibatch loss at step 500: 0.248951
Minibatch accuracy: 93.8%
Validation accuracy: 81.5%
M

## Problem 1
The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (nn.max_pool()) of stride 2 and kernel size 2.


* Both of the approaches (strides and pooling) reduce the dimensionality of the input (for strides/pooling size > 1). This by itself is a good thing because it reduces the computation time, number of parameters and allows to prevent overfitting.

* Reducing the input dimensionality is something you want because it forces the network to project its learned representations in a different and with lower dimensionality space. This is good computationally speaking because you have to allocate less memory and thus you can have bigger batches. But it's also something desirable because usually high-dimensional spaces have a lot of redundancy and are spaces in which all abjects appears to be sparse and dissimilar

* you can think about strides as downsampling the result of the 1-strided convolution by just taking every s-th result. max-pooling downsamples the result by taking the maximum number from a hypercube. If some important feature has been found, max-pool preserves it regardless of its position

* maxpooling forces the network to learn particular features, depending on the type of pooling you apply.

* The function you decide to use for the pooling operation, moreover, can force the network to give more importance to some features.

* Max-pooling, for instance, is widely used because allow the network to be robust to small variations of the input image.

* What happens, in practice, it that only the features with the highest activations pass through the max-pooling gate. If the input image is shifted by a small amount, then the max-pooling op produces the same output although the input is shifted (the maximum shift is thus equal to the kernel size).

* CNN without pooling are also capable of learning this kind of features, but with a bigger cost in term of parameters and computing time 



In [7]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():
    
  # Random Seed  
  tf.set_random_seed(42)   

  # Input data.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    pool = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    
    conv = tf.nn.conv2d(pool, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    pool = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    
    
    shape = pool.get_shape().as_list()
    reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [8]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.042297
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 50: 2.041883
Minibatch accuracy: 12.5%
Validation accuracy: 28.9%
Minibatch loss at step 100: 0.750566
Minibatch accuracy: 68.8%
Validation accuracy: 67.2%
Minibatch loss at step 150: 1.434590
Minibatch accuracy: 56.2%
Validation accuracy: 67.2%
Minibatch loss at step 200: 1.235511
Minibatch accuracy: 68.8%
Validation accuracy: 74.1%
Minibatch loss at step 250: 0.669600
Minibatch accuracy: 81.2%
Validation accuracy: 78.4%
Minibatch loss at step 300: 1.134165
Minibatch accuracy: 75.0%
Validation accuracy: 80.5%
Minibatch loss at step 350: 0.287982
Minibatch accuracy: 87.5%
Validation accuracy: 80.1%
Minibatch loss at step 400: 0.519595
Minibatch accuracy: 81.2%
Validation accuracy: 80.9%
Minibatch loss at step 450: 0.384158
Minibatch accuracy: 93.8%
Validation accuracy: 82.1%
Minibatch loss at step 500: 0.237474
Minibatch accuracy: 100.0%
Validation accuracy: 81.8%


# Problem 2

Try to get the best performance you can using a convolutional net. Look for example at the classic LeNet5 architecture, adding Dropout, and/or adding learning rate decay.

Reference : https://smist08.wordpress.com/2016/10/04/the-road-to-tensorflow-part-10-more-on-optimization/

https://github.com/tensorflow/tensorflow/blob/7e3b8b23835ab0ac55d390aed2349af6e05dbe3b/tensorflow/models/image/mnist/convolutional.py#L216


In [9]:
batch_size = 64
patch_size = 5
depth = 32
num_hidden = 64
SEED=42

graph = tf.Graph()

with graph.as_default():
    
  # Random Seed  
  tf.set_random_seed(42)   

  # Input data.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1,seed=SEED))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1,seed=SEED))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1,seed=SEED))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1,seed=SEED))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    pool = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    
    conv = tf.nn.conv2d(pool, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    pool = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    
    
    shape = pool.get_shape().as_list()
    reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    hidden = tf.nn.dropout(hidden, 0.5, seed=SEED)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  train_size = train_labels.shape[0]  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

  # L2 regularization for the fully connected parameters.
  regularizers = (tf.nn.l2_loss(layer1_weights) + tf.nn.l2_loss(layer1_biases) +
                  tf.nn.l2_loss(layer2_weights) + tf.nn.l2_loss(layer2_biases) +
                  tf.nn.l2_loss(layer3_weights) + tf.nn.l2_loss(layer3_biases) +
                  tf.nn.l2_loss(layer4_weights) + tf.nn.l2_loss(layer4_biases))
  # Add the regularization term to the loss.
  loss += 5e-4 * regularizers

  # Optimizer: set up a variable that's incremented once per batch and
  # controls the learning rate decay.
  batch = tf.Variable(0)
  # Decay once per epoch, using an exponential schedule starting at 0.01.
  learning_rate = tf.train.exponential_decay(
      0.01,                # Base learning rate.
      batch * batch_size,  # Current index into the dataset.
      train_size,          # Decay step.
      0.95,                # Decay rate.
      staircase=True)
  # Use simple momentum for the optimization.
  optimizer = tf.train.MomentumOptimizer(learning_rate,
                                         0.9).minimize(loss,
                                                       global_step=batch)
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [10]:
num_steps = 1001
num_epochs = 10

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  #for step in range(num_steps):
  for step in xrange(int(num_epochs * train_size) // batch_size):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 6.820992
Minibatch accuracy: 12.5%
Validation accuracy: 10.4%
Minibatch loss at step 50: 2.522034
Minibatch accuracy: 9.4%
Validation accuracy: 11.6%
Minibatch loss at step 100: 2.182222
Minibatch accuracy: 29.7%
Validation accuracy: 29.0%
Minibatch loss at step 150: 1.836490
Minibatch accuracy: 40.6%
Validation accuracy: 48.1%
Minibatch loss at step 200: 1.434090
Minibatch accuracy: 57.8%
Validation accuracy: 55.1%
Minibatch loss at step 250: 1.234791
Minibatch accuracy: 64.1%
Validation accuracy: 62.7%
Minibatch loss at step 300: 1.137870
Minibatch accuracy: 70.3%
Validation accuracy: 64.9%
Minibatch loss at step 350: 1.573621
Minibatch accuracy: 60.9%
Validation accuracy: 65.8%
Minibatch loss at step 400: 1.163043
Minibatch accuracy: 70.3%
Validation accuracy: 67.7%
Minibatch loss at step 450: 1.134970
Minibatch accuracy: 73.4%
Validation accuracy: 70.9%
Minibatch loss at step 500: 1.389763
Minibatch accuracy: 67.2%
Validation accuracy: 71.8%
Mi

Validation accuracy: 85.1%
Minibatch loss at step 4550: 0.770730
Minibatch accuracy: 84.4%
Validation accuracy: 85.3%
Minibatch loss at step 4600: 0.766865
Minibatch accuracy: 81.2%
Validation accuracy: 85.1%
Minibatch loss at step 4650: 1.008426
Minibatch accuracy: 79.7%
Validation accuracy: 84.3%
Minibatch loss at step 4700: 0.604522
Minibatch accuracy: 89.1%
Validation accuracy: 84.7%
Minibatch loss at step 4750: 0.753809
Minibatch accuracy: 82.8%
Validation accuracy: 85.0%
Minibatch loss at step 4800: 0.598319
Minibatch accuracy: 87.5%
Validation accuracy: 85.3%
Minibatch loss at step 4850: 0.713611
Minibatch accuracy: 84.4%
Validation accuracy: 84.9%
Minibatch loss at step 4900: 0.639198
Minibatch accuracy: 87.5%
Validation accuracy: 85.0%
Minibatch loss at step 4950: 0.759744
Minibatch accuracy: 82.8%
Validation accuracy: 85.2%
Minibatch loss at step 5000: 0.926995
Minibatch accuracy: 79.7%
Validation accuracy: 84.9%
Minibatch loss at step 5050: 0.587755
Minibatch accuracy: 90.6%

Minibatch loss at step 9050: 0.809945
Minibatch accuracy: 85.9%
Validation accuracy: 86.4%
Minibatch loss at step 9100: 0.583206
Minibatch accuracy: 89.1%
Validation accuracy: 86.6%
Minibatch loss at step 9150: 0.655513
Minibatch accuracy: 87.5%
Validation accuracy: 86.6%
Minibatch loss at step 9200: 0.642324
Minibatch accuracy: 85.9%
Validation accuracy: 86.1%
Minibatch loss at step 9250: 0.908809
Minibatch accuracy: 79.7%
Validation accuracy: 86.6%
Minibatch loss at step 9300: 1.052631
Minibatch accuracy: 79.7%
Validation accuracy: 86.8%
Minibatch loss at step 9350: 0.805621
Minibatch accuracy: 81.2%
Validation accuracy: 87.0%
Minibatch loss at step 9400: 0.743485
Minibatch accuracy: 82.8%
Validation accuracy: 86.7%
Minibatch loss at step 9450: 0.728669
Minibatch accuracy: 81.2%
Validation accuracy: 86.8%
Minibatch loss at step 9500: 0.618219
Minibatch accuracy: 85.9%
Validation accuracy: 87.1%
Minibatch loss at step 9550: 0.502564
Minibatch accuracy: 89.1%
Validation accuracy: 86.6%

Validation accuracy: 87.5%
Minibatch loss at step 13550: 0.623969
Minibatch accuracy: 84.4%
Validation accuracy: 87.4%
Minibatch loss at step 13600: 0.643113
Minibatch accuracy: 85.9%
Validation accuracy: 87.7%
Minibatch loss at step 13650: 0.674235
Minibatch accuracy: 87.5%
Validation accuracy: 87.9%
Minibatch loss at step 13700: 0.425816
Minibatch accuracy: 93.8%
Validation accuracy: 87.4%
Minibatch loss at step 13750: 0.550069
Minibatch accuracy: 89.1%
Validation accuracy: 87.5%
Minibatch loss at step 13800: 0.652499
Minibatch accuracy: 85.9%
Validation accuracy: 87.9%
Minibatch loss at step 13850: 0.554671
Minibatch accuracy: 89.1%
Validation accuracy: 87.8%
Minibatch loss at step 13900: 0.393839
Minibatch accuracy: 93.8%
Validation accuracy: 87.8%
Minibatch loss at step 13950: 0.389491
Minibatch accuracy: 96.9%
Validation accuracy: 87.8%
Minibatch loss at step 14000: 0.503865
Minibatch accuracy: 93.8%
Validation accuracy: 87.2%
Minibatch loss at step 14050: 0.661152
Minibatch accu

Minibatch loss at step 18000: 0.449408
Minibatch accuracy: 92.2%
Validation accuracy: 88.1%
Minibatch loss at step 18050: 0.452329
Minibatch accuracy: 90.6%
Validation accuracy: 88.3%
Minibatch loss at step 18100: 0.545095
Minibatch accuracy: 85.9%
Validation accuracy: 88.6%
Minibatch loss at step 18150: 0.521841
Minibatch accuracy: 84.4%
Validation accuracy: 88.2%
Minibatch loss at step 18200: 0.485597
Minibatch accuracy: 87.5%
Validation accuracy: 88.3%
Minibatch loss at step 18250: 0.593543
Minibatch accuracy: 89.1%
Validation accuracy: 88.3%
Minibatch loss at step 18300: 0.943807
Minibatch accuracy: 78.1%
Validation accuracy: 88.3%
Minibatch loss at step 18350: 0.674581
Minibatch accuracy: 85.9%
Validation accuracy: 88.1%
Minibatch loss at step 18400: 0.464860
Minibatch accuracy: 89.1%
Validation accuracy: 88.5%
Minibatch loss at step 18450: 0.574255
Minibatch accuracy: 85.9%
Validation accuracy: 88.0%
Minibatch loss at step 18500: 0.385194
Minibatch accuracy: 92.2%
Validation accu

Validation accuracy: 88.6%
Minibatch loss at step 22500: 0.617421
Minibatch accuracy: 84.4%
Validation accuracy: 88.7%
Minibatch loss at step 22550: 0.385556
Minibatch accuracy: 90.6%
Validation accuracy: 88.5%
Minibatch loss at step 22600: 0.522090
Minibatch accuracy: 87.5%
Validation accuracy: 88.3%
Minibatch loss at step 22650: 0.253083
Minibatch accuracy: 95.3%
Validation accuracy: 88.8%
Minibatch loss at step 22700: 0.453622
Minibatch accuracy: 90.6%
Validation accuracy: 89.0%
Minibatch loss at step 22750: 0.446359
Minibatch accuracy: 90.6%
Validation accuracy: 88.6%
Minibatch loss at step 22800: 0.353073
Minibatch accuracy: 90.6%
Validation accuracy: 88.6%
Minibatch loss at step 22850: 0.506711
Minibatch accuracy: 85.9%
Validation accuracy: 88.5%
Minibatch loss at step 22900: 0.438081
Minibatch accuracy: 87.5%
Validation accuracy: 88.8%
Minibatch loss at step 22950: 0.382641
Minibatch accuracy: 90.6%
Validation accuracy: 88.5%
Minibatch loss at step 23000: 0.486381
Minibatch accu

Minibatch loss at step 26950: 0.506241
Minibatch accuracy: 90.6%
Validation accuracy: 89.0%
Minibatch loss at step 27000: 0.383298
Minibatch accuracy: 90.6%
Validation accuracy: 89.0%
Minibatch loss at step 27050: 0.350894
Minibatch accuracy: 95.3%
Validation accuracy: 89.2%
Minibatch loss at step 27100: 0.435818
Minibatch accuracy: 89.1%
Validation accuracy: 89.0%
Minibatch loss at step 27150: 0.410724
Minibatch accuracy: 90.6%
Validation accuracy: 89.1%
Minibatch loss at step 27200: 0.373583
Minibatch accuracy: 90.6%
Validation accuracy: 89.3%
Minibatch loss at step 27250: 0.407634
Minibatch accuracy: 90.6%
Validation accuracy: 89.0%
Minibatch loss at step 27300: 0.493149
Minibatch accuracy: 89.1%
Validation accuracy: 89.0%
Minibatch loss at step 27350: 0.482392
Minibatch accuracy: 85.9%
Validation accuracy: 88.9%
Minibatch loss at step 27400: 0.510692
Minibatch accuracy: 90.6%
Validation accuracy: 89.1%
Minibatch loss at step 27450: 0.317950
Minibatch accuracy: 92.2%
Validation accu